In [ ]:
#bootstrap function 
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pickle
from matplotlib.ticker import FuncFormatter, MaxNLocator
matplotlib.rcParams.update({'font.size': 16})

def bootstrap(data1, data2, num_samples=10000000, alpha=0.05):
    n = len(data1)
    idx = np.random.randint(0, n, (num_samples, n))
    samples1 = data1[idx]
    samples2 = data2[idx]

    stat =np.mean(samples1,1)/np.mean(samples2,1)

    return (stat[int((alpha/2.0)*num_samples)],
            stat[int((1-alpha/2.0)*num_samples)])

In [ ]:
#matching plot
bootstrap_num = 100000
data = pickle.load(open('./results/','rb'))

gaps = set()
for trial_data in data:
    thetastar = trial_data['thetastar']
    max_val = np.max(thetastar)
    for i in range(len(thetastar)):
        if thetastar[i] != 0 and thetastar[i] != max_val:
            gap = max_val - thetastar[i]
            break
            
    gaps.add(gap)
    
gaps = sorted(list(gaps))

chen_ratio = []
chen_ratio_upper = []
chen_ratio_lower = []
unif_ratio_upper = []
unif_ratio_lower = []

for gap in gaps:
    perf_combi_alg = []
    perf_chen = []
    perf_unif = []
    combi_alg_correct = []
    
    for trial_data in data:
        thetastar = trial_data['thetastar']
        max_val = np.max(thetastar)
        go_forward = False
        for i in range(len(thetastar)):
            if thetastar[i] != 0 and thetastar[i] != max_val:
                if gap == max_val - thetastar[i]:
                    go_forward = True
                    break
                else:
                    break
                
        if go_forward:
            
            if 'num_samples_combi_no_torch' in trial_data:

                perf_combi_alg.append(trial_data['num_samples_combi_no_torch'])
                perf_chen.append(trial_data['num_samples_chen']) 
                combi_alg_correct.append(trial_data['correct_combi_no_torch'])
            if 'num_samples_uniform' in trial_data:
                perf_unif.append(trial_data['num_samples_uniform'])

    upper, lower = bootstrap(np.array(perf_chen), np.array(perf_combi_alg), num_samples=bootstrap_num)
    upper_unif, lower_unif = bootstrap(np.array(perf_unif), np.array(perf_combi_alg), num_samples=bootstrap_num)


    chen_ratio.append(np.mean(perf_chen)/np.mean(perf_combi_alg))
    chen_ratio_upper.append(upper)
    chen_ratio_lower.append(lower)
    unif_ratio_upper.append(upper_unif)
    unif_ratio_lower.append(lower_unif)
    


    
    print(f"gap {gap} chen_sample_comp {np.mean(perf_chen)}")
    print(f"gap {gap} combi_sample_comp {np.mean(perf_combi_alg)}")
    print(f"prob correct {np.mean(combi_alg_correct)}")

print(f"chen_ratio {chen_ratio}")
print(f"unif_ratio_lower {unif_ratio_lower}")

chen_ratio_upper = np.array(chen_ratio_upper)
chen_ratio_lower = np.array(chen_ratio_lower)
chen_mean_perf = .5*(chen_ratio_upper+chen_ratio_lower)

plt.errorbar(gaps, chen_mean_perf, chen_ratio_upper-chen_ratio_lower, label = "CLUCB")


unif_ratio_upper = np.array(unif_ratio_upper)
unif_ratio_lower = np.array(unif_ratio_lower)
unif_mean_perf = .5*(unif_ratio_upper+unif_ratio_lower)

plt.errorbar(gaps, unif_mean_perf, unif_ratio_upper-unif_ratio_lower, label = "Uniform")

# plt.plot(gaps, chen_ratio, label = "CLUCB")
plt.legend(loc="upper left")
plt.xlabel("h")
plt.ylabel("Units of Average AE Sample Complexity")
plt.gca().invert_xaxis()

plt.show()


print(unif_ratio_upper)

In [ ]:
#shortest path plot
bootstrap_num = 1000000

data = pickle.load(open('./results/','rb'))

gaps = set()
for trial_data in data:
    thetastar = trial_data['thetastar']
    max_val = np.max(thetastar)
    for i in range(len(thetastar)):
        if thetastar[i] != -1 and thetastar[i] != max_val:
            gap = max_val - thetastar[i]
            break
            
    gaps.add(gap)
    
gaps = sorted(list(gaps))

chen_ratio = []
chen_ratio_upper = []
chen_ratio_lower = []
unif_ratio = []
unif_ratio_upper = []
unif_ratio_lower = []

for gap in gaps:
    perf_combi_alg = []
    perf_chen = []
    combi_alg_correct = []
    
    perf_uniform = []
    
    for trial_data in data:
        thetastar = trial_data['thetastar']
        max_val = np.max(thetastar)
        go_forward = False
        for i in range(len(thetastar)):
            if thetastar[i] != -1 and thetastar[i] != max_val:
                if gap == max_val - thetastar[i]:
                    go_forward = True
                    break
                else:
                    break
                
        if go_forward:
            if 'num_samples_chen' in trial_data:
                perf_combi_alg.append(trial_data['num_samples_combi_no_torch'])
                perf_chen.append(trial_data['num_samples_chen'])
                combi_alg_correct.append(trial_data['correct_combi_no_torch'])

            if 'num_samples_uniform' in trial_data:
                perf_uniform.append(trial_data['num_samples_uniform'])
            

    upper, lower = bootstrap(np.array(perf_chen), np.array(perf_combi_alg), num_samples=bootstrap_num)
    upper_unif, lower_unif = bootstrap(np.array(perf_uniform), np.array(perf_combi_alg), num_samples=bootstrap_num)


    chen_ratio.append(np.mean(perf_chen)/np.mean(perf_combi_alg))
    chen_ratio_upper.append(upper)
    chen_ratio_lower.append(lower)
    unif_ratio.append(np.mean(perf_uniform)/np.mean(perf_combi_alg))
    unif_ratio_upper.append(upper_unif)
    unif_ratio_lower.append(lower_unif)
    
    print(f"prob correct {np.mean(combi_alg_correct)}")
    print(f"gap {gap} chen_sample_comp {np.mean(perf_chen)}")
    print(f"gap {gap} combi_sample_comp {np.mean(perf_combi_alg)}")
    
print(f"chen_ratio {chen_ratio}")
print(f"uniform_ratio {unif_ratio}")

chen_ratio_upper = np.array(chen_ratio_upper)
chen_ratio_lower = np.array(chen_ratio_lower)
chen_mean_perf = .5*(chen_ratio_upper+chen_ratio_lower)

plt.errorbar(gaps, chen_mean_perf, chen_ratio_upper-chen_ratio_lower, label = "CLUCB")


unif_ratio_upper = np.array(unif_ratio_upper)
unif_ratio_lower = np.array(unif_ratio_lower)
unif_mean_perf = .5*(unif_ratio_upper+unif_ratio_lower)

plt.errorbar(gaps, unif_mean_perf, unif_ratio_upper-unif_ratio_lower, label = "Uniform")

# plt.plot(gaps, chen_ratio, label = "CLUCB")
plt.legend(loc="upper left")
plt.xlabel("h")
plt.gca().invert_xaxis()

plt.show()

In [ ]:
#biclique
import numpy as np
import matplotlib.pyplot as plt
import pickle

#matching plot

bootstrap_num = 1000000

data = pickle.load(open('./results/','rb'))

gaps = set()
for trial_data in data:
    thetastar = trial_data['thetastar']
    max_val = np.max(thetastar) 
    for i in range(len(thetastar)):
        if thetastar[i] != 0 and thetastar[i] != max_val:
            gap = max_val - thetastar[i]
            break
            
    gaps.add(gap)
    
gaps = sorted(list(gaps))    

chen_ratio = []
chen_ratio_upper = []
chen_ratio_lower = []
unif_ratio_upper = []
unif_ratio_lower = []
akshay_ratio_upper = []
akshay_ratio_lower = []
akshay_ratio = []

for gap in gaps:
    perf_combi_alg = []
    perf_chen = []
    perf_unif = []
    perf_akshay = []
    
    combi_alg_correct = []
    akshay_correct = []
    
    for trial_data in data:
        thetastar = trial_data['thetastar']
        max_val = np.max(thetastar)
        go_forward = False
        for i in range(len(thetastar)):
            if thetastar[i] != 0 and thetastar[i] != max_val:
                if gap == max_val - thetastar[i]:
                    go_forward = True
                    break
                else:
                    break
                
        if go_forward:
            if 'num_samples_combi_no_torch' in trial_data:
                perf_combi_alg.append(trial_data['num_samples_combi_no_torch'])
                perf_chen.append(trial_data['num_samples_chen']) 
                combi_alg_correct.append(trial_data['correct_combi_no_torch'])
                
            if 'num_samples_uniform' in trial_data:
                perf_unif.append(trial_data['num_samples_uniform'])
            
            if 'num_samples_akshay' in trial_data:
                perf_akshay.append(trial_data['num_samples_akshay'])
                akshay_correct.append(trial_data['correct_akshay'])

    upper, lower = bootstrap(np.array(perf_chen), np.array(perf_combi_alg), num_samples=bootstrap_num)
    upper_unif, lower_unif = bootstrap(np.array(perf_unif), np.array(perf_combi_alg), num_samples=bootstrap_num)
    upper_akshay, lower_akshay = bootstrap(np.array(perf_akshay), np.array(perf_combi_alg), num_samples=bootstrap_num)

    chen_ratio.append(np.mean(perf_chen)/np.mean(perf_combi_alg))
    chen_ratio_upper.append(upper)
    chen_ratio_lower.append(lower)
    unif_ratio_upper.append(upper_unif)
    unif_ratio_lower.append(lower_unif)
    akshay_ratio_upper.append(upper_akshay)
    akshay_ratio_lower.append(lower_akshay)
    akshay_ratio.append(np.mean(perf_akshay)/np.mean(perf_combi_alg))

    
    print(f"gap {gap} chen_sample_comp {np.mean(perf_chen)}")
    print(f"gap {gap} combi_sample_comp {np.mean(perf_combi_alg)}")
    print(f"combi alg prob correct {np.mean(combi_alg_correct)}")
    print(f"akshay prob correct {np.mean(akshay_correct)}")

print(f"chen_ratio {chen_ratio}")
print(f"unif_ratio_lower {unif_ratio_lower}")
print(f"akshay_ratio_lower {akshay_ratio_lower}")
print(f"akshay_ratio {akshay_ratio}")


#plot chen
chen_ratio_upper = np.array(chen_ratio_upper)
chen_ratio_lower = np.array(chen_ratio_lower)
chen_mean_perf = .5*(chen_ratio_upper+chen_ratio_lower)
plt.errorbar(gaps, chen_mean_perf, chen_ratio_upper-chen_ratio_lower, label = "CLUCB")

#plot uniform
unif_ratio_upper = np.array(unif_ratio_upper)
unif_ratio_lower = np.array(unif_ratio_lower)
unif_mean_perf = .5*(unif_ratio_upper+unif_ratio_lower)
plt.errorbar(gaps, unif_mean_perf, unif_ratio_upper-unif_ratio_lower, label = "Uniform")

#plot akshay
akshay_ratio_upper = np.array(akshay_ratio_upper)
akshay_ratio_lower = np.array(akshay_ratio_lower)
akshay_mean_perf = .5*(akshay_ratio_upper+akshay_ratio_lower)
plt.errorbar(gaps, akshay_mean_perf, akshay_ratio_upper-akshay_ratio_lower, label = "DisRegion")

plt.plot(gaps,np.ones_like(gaps), label = "Peace")


# plt.plot(gaps, chen_ratio, label = "CLUCB")
plt.legend(loc="upper left")
plt.xlabel("h")
plt.ylabel("Relative Sample Complexity")
plt.gca().invert_xaxis()

plt.show()

print(unif_ratio_upper)

In [ ]:
# transductive experiments
import pickle
import matplotlib.pyplot as plt
outputs = pickle.load(open('./transductive.pkl','rb'))
outputs1 = outputs[0]
outputs2 = outputs[1]
outputs3 = outputs[2]
outputs4 = outputs[3]

num_samples_rage = []
num_samples_linalg = []

rage_means = []
rage_error = []
linalg_means = []
linalg_error = []

for o in outputs1:
    num_samples_rage.append(o['num_samples_rage'])
    num_samples_linalg.append(o['num_samples_linalg'])
print(np.mean(num_samples_rage), np.mean(num_samples_linalg))
rage_means.append(np.mean(num_samples_rage))
rage_error.append(np.std(num_samples_rage)/np.sqrt(20))
linalg_means.append(np.mean(num_samples_linalg))
linalg_error.append(np.std(num_samples_linalg)/np.sqrt(20))

num_samples_rage = []
num_samples_linalg = []
for o in outputs2:
    num_samples_rage.append(o['num_samples_rage'])
    num_samples_linalg.append(o['num_samples_linalg'])
print(np.mean(num_samples_rage), np.mean(num_samples_linalg))
rage_means.append(np.mean(num_samples_rage))
rage_error.append(np.std(num_samples_rage)/np.sqrt(20))
linalg_means.append(np.mean(num_samples_linalg))
linalg_error.append(np.std(num_samples_linalg)/np.sqrt(20))

num_samples_rage = []
num_samples_linalg = []
for o in outputs3:
    num_samples_rage.append(o['num_samples_rage'])
    num_samples_linalg.append(o['num_samples_linalg'])
print(np.mean(num_samples_rage), np.mean(num_samples_linalg))
rage_means.append(np.mean(num_samples_rage))
rage_error.append(np.std(num_samples_rage)/np.sqrt(20))
linalg_means.append(np.mean(num_samples_linalg))
linalg_error.append(np.std(num_samples_linalg)/np.sqrt(20))
print(num_samples_rage, num_samples_linalg)
print(np.std(num_samples_rage), np.std(num_samples_linalg))

num_samples_rage = []
num_samples_linalg = []
for o in outputs4:
    num_samples_rage.append(o['num_samples_rage'])
    num_samples_linalg.append(o['num_samples_linalg'])
print(np.mean(num_samples_rage), np.mean(num_samples_linalg))
rage_means.append(np.mean(num_samples_rage))
rage_error.append(np.std(num_samples_rage)/np.sqrt(20))
linalg_means.append(np.mean(num_samples_linalg))
linalg_error.append(np.std(num_samples_linalg)/np.sqrt(20))

plt.errorbar([1e3, 1e4, 1e5, 1e6], rage_means, yerr=rage_error, label='Rage')
plt.errorbar([1e3, 1e4, 1e5, 1e6], linalg_means, yerr=linalg_error, label='Love')
plt.xscale('log')
plt.legend()
plt.show()

In [ ]:
# fixed budget experiments
import pickle
import matplotlib.pyplot as plt
import numpy as np
data = pickle.load(open('fixed_budget.pkl','rb'))

data_uniform = []
data_succ_elim = []

for d in data:
    d_uniform = d['uniform']
    d_succ_elim = d['succ_elim']
    Ts = sorted(d_uniform.keys())
    data_uniform.append([d_uniform[t] for t in Ts])
    data_succ_elim.append([d_succ_elim[t] for t in Ts])

data_uniform = np.array(data_uniform)
data_succ_elim = np.array(data_succ_elim)

plot_uniform_mean = np.mean(np.array(data_uniform), axis=0)
plot_uniform_std = np.std(np.array(data_uniform), axis=0)/np.sqrt(data_uniform.shape[0])

plot_succ_elim_mean = np.mean(np.array(data_succ_elim), axis = 0)
plot_succ_elim_std = np.std(np.array(data_succ_elim), axis = 0)/np.sqrt(data_succ_elim.shape[0])

plt.title('Experimental Design')
plt.errorbar(Ts, plot_uniform_mean, plot_uniform_std, label='Uniform Sampling')
plt.errorbar(Ts, plot_succ_elim_mean, plot_succ_elim_std, label='Finite Rage')
plt.xlabel('T:Number of sample')
plt.ylabel('Probability of Success')
plt.legend